# Install Packages

In [ ]:
!pip install nltk
!pip install wordcloud


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix
try:
    from sklearn.preprocessing import OrdinalEncoder
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

import nltk
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Data Cleaning

In [2]:
tweets=pd.read_csv('judge-1377884607_tweet_product_company.csv', encoding = 'unicode_escape')
tweets.tweet_text=tweets.tweet_text.map(lambda x: str(x))
tweets = tweets[tweets.is_there_an_emotion_directed_at_a_brand_or_product != 'No emotion toward brand or product']  
tweets = tweets[tweets.is_there_an_emotion_directed_at_a_brand_or_product != "I can't tell"]
tweets=tweets.replace('Negative emotion', 0)
tweets=tweets.replace('Positive emotion', 1)
tweets.is_there_an_emotion_directed_at_a_brand_or_product=pd.to_numeric(tweets.is_there_an_emotion_directed_at_a_brand_or_product)

tweets.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,1
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,1
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,1


# Create Count Vectorizer Dataframe

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1), tokenizer = token.tokenize)
text_counts= cv.fit_transform(tweets['tweet_text'])

In [4]:
df = pd.DataFrame(text_counts.todense(),columns = cv.get_feature_names())

# Train Test Split

In [5]:
X=df
y=tweets['is_there_an_emotion_directed_at_a_brand_or_product']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

# CV Gridsearch Log Reg Model

In [ ]:
# Grid Search

param_grid = { 
    'C': list(range(1,25)),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None]
    }


grid_lr=GridSearchCV(LogisticRegression(random_state=1), param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

grid_lr.fit(X_train, y_train)

grid_lr.best_params_

# Create Log Reg Model

In [13]:
lr = LogisticRegression(class_weight=None, C=1, penalty='l1')

lr.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Predict

In [14]:
lr_preds = lr.predict(X_test)

lr_f1 = metrics.f1_score(y_test, lr_preds)
lr_acc = metrics.accuracy_score(y_test, lr_preds)
lr_pre = metrics.precision_score(y_test, lr_preds)

print(lr_f1, lr_acc, lr_pre)

0.9178617992177315 0.8579481397970687 0.8877679697351829
